# Quickstart

### Intro

In this quick tutorial, we create a database of orders and a database of time series. For each order we then retrieve the corresponding market infromation from the timeseries and we add them to the orders database. This allows us to introduce some methods of the database api. If you are woriking locally, please verify that a mongodb 3.4 is active on your machine.

### Database Setup

First of all we import the database api. We connect to the port `localhost:27017` and to the databases `orders_example` and `ts_example`.

In [1]:
import time
import os
import sys
sys.path.insert(0, os.path.abspath('../dana'))
import dana
api = dana.Dana(engine_url='localhost:27017', db_orders_name='orders_example', db_ts_name='ts_example')

We use the api methods `drop_ts()` and `drop_orders()` to drop the orders and ts databases, if they exist.

In [27]:
t0 = time.time()
api.drop_ts()
api.drop_orders()
print('Database cleaned in seconds: ' + str(time.time()-t0))

Database cleaned in seconds: 0.03942298889160156


### Market parameters

We consider an hipoetetical market composed by the symbols AAPL, MSFT and GOOG, that is open in the Julian dates from 200000 to 200010 and that opens at 100 and closes at 200 minutes from midnight. Each day for each symbol 10 random orders are traded.

In [28]:
date_list = range(200000, 200010)
symbol_list = ['AAPL', 'MSFT', 'GOOG']
market_open = 100
market_close = 200
n_orders = 10

### Generate Orders

We define a function that generate an order with random attributes. We use this function to generate orders for the market symbols and dates. The orders start and end at a random times between the market open and the market close. For each order, the traded volume and the execution price are random quantities. Once generated, we insert them in the database using the api method `insert_orders()`.

In [29]:
import numpy as np
import itertools

def generate_order(symbol, date, min_start, min_end):

    """ Generate a random order """

    # maximum volume of an order
    v_max = 100

    # maximum number of trades per order
    n_max = 10

    # random start and end time
    t_random = [0, 0]
    while t_random[0] == t_random[1]:
        t_random = np.random.randint(min_start, min_end, 2)

    order = {
        'mgr': 'mgr_test',
        'bkr': 'bkr_test',
        'symbol': symbol,
        'sign': 1 if np.random.rand() < 0.5 else -1,
        'date': date,
        'min_start': int(str(min(t_random))),
        'min_end': int(str(max(t_random))),
        'volume': np.random.randint(0, v_max),
        'price': 100. + 10.*np.random.rand(),
        'ntrades': np.random.randint(0, n_max),
        'is_ok': False
    }

    return order

orders_list = []
for symbol, date in itertools.product(symbol_list, date_list):
    orders_list = orders_list + [generate_order(symbol, date, market_open, market_close) for i in range(n_orders)]

t0 = time.time()
api.insert_orders(orders_list)
print('Insert ' + str(len(orders_list)) + ' orders in: ' + str(time.time() - t0))

Insert 300 orders in: 0.4813880920410156


### Generate timeseries

We generate the timeseries of traded prices and volumes with one minute frequency for the market symbols and dates. Each time series starts at the market open and ends at the market close and consists in a random walk starting at price 100. The volume traded during each minute is constant and equal to 100 volume units. We insert the time series in the database, using the api method `insert_ts()`. Once the timeseries are inserted in the database, we generate an index for all the symbols on the `date` field using the api method `ts_index()`. The index in the database allows to improve the speed in querying the database.

In [30]:
def generate_ts(date, min_start, min_end, volume):

    price = 100.
    ts = []
    for mins in range(min_start, min_end):
        record = {
            'date': date,
            'mins': mins,
            'price': price,
            'volume': volume
                 }
        ts.append(record)
        price += 2.*(np.random.rand()-0.5)

    return ts

t0 = time.time()
for symbol, date in itertools.product(symbol_list, date_list):
    ts = generate_ts(date=date, min_start=market_open, min_end=market_close, volume=100)
    api.insert_ts(symbol, ts)
print('Insert ' + str(len(symbol_list)*len(date_list)) + ' timeseries in seconds: ' + str(time.time() - t0))

t0 = time.time()
for symbol in symbol_list:
    api.ts_index(symbol=symbol)
print('Generated indexex in seconds: ' + str(time.time() - t0))

Insert 30 timeseries in seconds: 0.5881509780883789
Generated indexex in seconds: 0.017637968063354492


### Extract market info

For each order, we retrieve the corresponding market infromation, i.e.:
* the daily participation rate, defined as $V_{Order}/V_{Day}$
* the period pariticipaiton rate, defined as $V_{Order}/V_{Period}$
* the duration in volume time, defined as $V_{Period}/V_{Day}$
* the duration in physical time, defined as $t_{end}-t_{start}$
* the daily volatility, defined as $\sigma_D = \frac{p_{high} - p_{low}}{p_{open}}$
* the temporary impact, defined as $\epsilon \log p_{end}/p_{start}/ \sigma_D$
* the vwap impact,  defined as $\epsilon \log p_{vwap}/p_{start}/ \sigma_D$

This is done using the api method `assign_candle_orders()`.

In [31]:
t0 = time.time()
for symbol, date in itertools.product(symbol_list, date_list):
    api.assign_candle_orders(symbol=symbol, date=date)
print('Retrieved market information in seconds: ' + str(time.time() - t0))

Retrieved market information in seconds: 0.5459749698638916


We show one of the orders using the api method `get_orders()`:

In [32]:
api.get_orders(symbol='MSFT', date=200000)[0]

{'bkr': 'bkr_test',
 'candle_day': {'close': 98.07675675971257,
  'high': 105.29900874302302,
  'low': 95.67374030989704,
  'open': 100.0,
  'volume': 10000,
  'vwap': 100.57849870224912},
 'candle_period': {'close': 99.34189473162803,
  'high': 105.29900874302302,
  'low': 95.67374030989704,
  'open': 97.98998529847876,
  'volume': 8000,
  'vwap': 100.92283575677413},
 'date': 200000,
 'duration_ph': 80,
 'duration_vol': 0.8,
 'impact_end': 0.1423555111737901,
 'impact_vwap': 0.585558094846318,
 'is_ok': True,
 'mgr': 'mgr_test',
 'min_end': 192,
 'min_start': 112,
 'ntrades': 2,
 'pr_day': 0.0032,
 'pr_period': 0.004,
 'price': 103.67145592503103,
 'sign': 1,
 'symbol': 'MSFT',
 'volatility_day': 0.09625268433125982,
 'volume': 32}